In [2]:
import ROOT
import uproot
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak

from sklearn.preprocessing import normalize as norm

Welcome to JupyROOT 6.30/04


In [5]:
FILE_PATH = "../Dati/Run_Na_Couples_CDF20_Data_5_9_2024_Ascii.root" #Relative file path
file = uproot.open(FILE_PATH)
file["tree"].show()


baselines = np.array(file["tree"]["base"].array()) 
charges = np.array(file["tree"]["charge"].array())
amp = np.array(file["tree"]["amp"].array())
lead_edge = np.array(file["tree"]["letime"].array())
t_edge = np.array(file["tree"]["tetime"].array())
channel= np.array(file["tree"]["ch"].array())
waveform = np.array(file["tree"]["pshape"].array())

nch=file["tree"]["ch"].array()
print(nch)
N_ev=len(charges[:, 0])
print(N_ev)

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
ev                   | int32_t                  | AsDtype('>i4')
nch                  | int32_t                  | AsDtype('>i4')
ch                   | int32_t[]                | AsJagged(AsDtype('>i4'))
base                 | float[]                  | AsJagged(AsDtype('>f4'))
amp                  | float[]                  | AsJagged(AsDtype('>f4'))
charge               | float[]                  | AsJagged(AsDtype('>f4'))
letime               | float[]                  | AsJagged(AsDtype('>f4'))
tetime               | float[]                  | AsJagged(AsDtype('>f4'))
ratecount            | float[]                  | AsJagged(AsDtype('>f4'))
pshape               | float[][1024]            | AsJagged(AsDtype("('>f4', (...
[[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], ..., [4, 5, 6, 7, ..., 13, 14, 15]]
100000


In [6]:
print(charges.shape)

(100000, 12)


In [ ]:
def set_flag():
    flag=np.zeros((N_ev, 2))
    
    for i in range(N_ev):
    flag[i][0]=np.where( "condition" )[0][0]
    flag[i][1]=np.where( "condition"  )[0][1]

    return(flag)

#posso controllare come un canale è accoppiato agli altri
ch=0
msk=np.where(flag[:,0]== ch))[0]
plt.hist(flag[msk, :])

#

ad ogni event è associata una flag, ora devo selezionare tutti gli event con una stessa flag

In [8]:
#gli elementi della coppia vanno messi in ordine crescente
couple=[ , ]
idx=np.where((flag[:,0]==couple[0] ) & (flag[:,1]==couple[1]))[0]
paired_wf_1=waveform[couple[0]][idx]
paired_wf_2=waveform[couple[1]][idx]

0.0


: 

In [ ]:
def timing_measurement(paired_wf_1, sampling, shiftR, shiftL, n_mean, thresh):
    wf_1_len = len(paired_wf_1)
    new_wf_1 = np.array(paired_wf_1).reshape([np.array(paired_wf_1).shape[0], 1, np.array(paired_wf_1).shape[1]])

    for evt in range(new_wf_1.shape[0]):
    for ch in range(new_wf_1.shape[1]):
        bsl_1 = new_wf_1[evt, ch, 0:32].mean()
        new_wf_1[evt, ch, :] = np.convolve(new_wf_1[evt, ch], [1/n] * n, mode = "same") - bsl_1

    new_wf_1 = -norm(-new_wf_1.reshape(-1, 1024), norm = "max", axis = 1, copy = True, return_norm = False)
    threshold = thr

    ledges1 = np.zeros(new_wf_1.shape[0])

    for evt in range(new_wf_1.shape[0]):
    ledge= np.where(np.diff(np.sign(new_wf_1[evt] - threshold)))[0][0]
    coef = np.polyfit(new_wf_1[evt][ledge-shiftL:ledge+shiftR],np.arange(ledge-shiftL, ledge+shiftR),1)
    ledges1[evt] = np.poly1d(coef)(threshold) 

return ledges1*sampling

In [ ]:
#data una coppia e un dataset la funzione ritorna l'offset relativo alla misura di timing
def offset_and_resolution_estimation( couple, flag, waveform ):
    idx=np.where((flag[:,0]==couple[0] ) & (flag[:,1]==couple[1]))[0]
    paired_wf_1=waveform[couple[0]][idx]
    paired_wf_2=waveform[couple[1]][idx]
    timeres=timing_measurement(paired_wf_1)-timing_measurement(paired_wf_2)
    return(np.mean(timeres), np.std(timeres), timeres)


In [ ]:
def offet_matrix_filling(nch, flag, waveform):
    offset_matrix=np.zeros(nch, nch)
    for i in range(nch):
        for j in range(i):
            offset_matrix[i][j]=offset_and_resolution_estimation([i,j], flag, waveform)
            offset_matrix[j][i]=-offset_matrix[i][j]
    return(offset_matrix)

def offset_matrix_save(offset_matrix):

def offset_matrix_load(filename)

In [ ]:
[couple_1, couple_2]